In [1]:
%load_ext autoreload

In [2]:
import pandas as pd
from os import path
import glob
from itertools import chain
import warnings
from scrapers.scrapepool import ScrapePool
from scrapers.afnor import AfnorScraper

# disable warnings, because pandas throws warnings for mixed type columns all the time..
warnings.filterwarnings('ignore')

%autoreload

In [3]:
workspace = '.\\data\\afnor'
write_dir = 'out'
file_found = 'found.csv'
file_extended = 'perinorm.csv'
col_found_id = 'Official identifier'
col_extended_id = 'Dokumentnummer'
search_for_document_families = False
search_batch_size = 500
num_threads = 10

In [4]:
# scraper settings
args = {
    'phantomjs_exec': 'C:\\phantomjs.exe',
    'save_to': '.\\data\\afnor\\out',
    'user': 'knut.blind@tu-berlin.de',
    'password': 'kblind',
    'urls': {
        'base': "https://sagaweb.afnor.org",
        'results': "/en-US/sw/Recherche/Resultat/1/?offset=10000&selectall=false",
        'logout': "/en-US/sw/Identification/Deconnexion"
    },
    'states': AfnorScraper.STATES_FIND_ALL
}

In [5]:
def refresh(target):
    print(">> refreshing...")
    inputs = glob.glob(path.join(workspace, write_dir, "*.csv"))
    inputs.append(path.join(workspace, file_found))
    df = pd.concat((pd.read_csv(f, sep=";", quotechar="\"", encoding="ISO-8859-1") for f in inputs), axis=0, ignore_index=True)
    found = set(df[col_found_id])
    searched_for = set(["_".join(".".join(path.basename(f).split(".")[:-1]).split("_")[:-1]).replace('#', '/').replace('=',':') for f in glob.glob(path.join(workspace, write_dir, "*.*"))])
    missing = (target - found) - searched_for
    print(">> target: " + str(len(target)) + ", found: " + str(len(found)) + ", missing: " + str(len(missing)))
    
    return target, found, missing

In [6]:
extended = pd.read_csv(path.join(workspace, file_extended), sep=";", quotechar="\"", encoding="ISO-8859-1")
target = set(chain.from_iterable([ref.split("*") for ref in extended[col_extended_id].tolist()]))
# manipulate keywords to search for document families
if search_for_document_families:
    target = set([k.split("/")[0].split("-")[0] for k in target])

In [7]:
_, found, missing = refresh(target)

last_missing = None

# make sure we're not repeatedly looking for the same thing
while len(missing) > 0 and last_missing != missing:
    
    last_missing = set(missing)
    
    # Always only look for first n missing standards, then refresh keyword-list.
    # This prevent searches for standards that have already been found while looking for other standards.
    keywords = list(missing)[:min(len(missing), search_batch_size)]
    
    # output
    first_next = min(len(keywords), 3)
    second_next = len(keywords) - first_next
    print(">> looking for: " + ", ".join(keywords[:first_next]) + " (" + str(second_next) + " more)")
    
    # run scrapers
    s = ScrapePool(AfnorScraper, keywords, args, chunk_size=round(len(keywords)/num_threads))
    s.run()
    
    _, found, missing = refresh(target)

>> refreshing...
>> target: 89226, found: 60275, missing: 4966
>> looking for: NF F42-328, NF P61-534-8, NF C42-161/A2 (497 more)


connection tested successfully!
connection tested successfully!
connection tested successfully!
connection tested successfully!
connection tested successfully!
connection tested successfully!
connection tested successfully!
connection tested successfully!
connection tested successfully!
connection tested successfully!
logging in
logging in
logging in
logging in
searching for PR NF E29-674-1
searching for NF X43-539
searching for NF F42-328
searching for NF A32-301/A1
logging in
logging in
logging in
logging in
searching for NF C47-742/A1
searching for NF E86-200-3/A4
searching for NF EN 50329/A1
searching for NF P61-534-8
logging in
logging in
logging in
searching for NF Z82-185-5
searching for NF C48-205F13
searching for NF C42-161/A2
logging in
logging in
logging in
searching for PR NF X33-026-4
logging in
searching for NF U42-154
searching for PR NF P20-702-1
searching for NF R13-010-5
logging in
searching for NF P98-906-1
logging in
logging in
logging in
logging in
searching for NF

searching for NF E10-112-2
searching for NF C97-190
logging in
logging in
logging in
searching for PR NF C56-282-6-1
searching for NF L24-481
logging in
searching for NF C93-040
searching for NF C44-056-6-1
logging in
logging in
logging in
logging in
searching for NF C32-102-1F1
searching for NF P98-818-42
searching for NF C26-308
searching for NF E29-735
logging in
logging in
logging in
searching for PR NF C46-611-2
searching for NF C71-017
searching for NF L13-221
logging in
logging in
logging in
logging in
searching for NF C73-839/A1
searching for NF H90-212
searching for NF C93-537-1-17
searching for NF C92-507
logging in
searching for NF S76-007-7
logging in
logging in
logging in
logging in
searching for NF C26-158-2
searching for NF T51-227-2
searching for NF L10-548
logging in
searching for NF L40-050
logging in
logging in
logging in
searching for NF E90-972-4
searching for NF P14-205-2
searching for NF T94-214
searching for NF C97-165
logging in
logging in
logging in
logging in

logging in
searching for NF C46-952
searching for NF P12-521-1
logging in
logging in
searching for NF Z83-586
searching for NF E51-290-2
logging in
logging in
searching for NF P98-344
searching for NF E01-201-4
logging in
searching for NF P19-125
logging in
logging in
searching for NF P52-721-2
searching for NF Z70-083-2
logging in
searching for NF E66-314
logging in
searching for NF C93-563-2-4
logging in
logging in
searching for NF S52-388-10
logging in
searching for NF P98-916-2
logging in
searching for NF S57-036-2
searching for NF E51-772-4
logging in
logging in
searching for NF C93-919-3-5
searching for NF E37-305
logging in
logging in
searching for NF C72-222/A2
searching for NF B30-111
logging in
searching for NF C17-151-7
logging in
searching for NF M07-119-3
logging in
logging in
searching for PR NF G38-176
searching for NF A50-420
logging in
logging in
searching for NF T72-072
searching for NF C27-261-2
logging in
searching for NF T51-603-2
logging in
logging in
searching fo

>> refreshing...
>> target: 89226, found: 60277, missing: 4466
>> looking for: NF A51-307, NF E90-327, NF X43-024 (497 more)


...

>> refreshing...
>> target: 89226, found: 60307, missing: 466
>> looking for: NF C99-126, NF S12-111, PR NF E05-031-607 (463 more)


connection tested successfully!
connection tested successfully!
connection tested successfully!
connection tested successfully!
connection tested successfully!
connection tested successfully!
connection tested successfully!
connection tested successfully!
connection tested successfully!
connection tested successfully!
logging in
logging in
logging in
logging in
searching for NF C91-147-1
searching for PR NF U66-069/A1
searching for NF C52-216-7
searching for NF C99-126
logging in
logging in
logging in
logging in
searching for NF L10-535
searching for NF S12-111
searching for NF L52-114-009
searching for NF EN 300676-2
logging in
logging in
logging in
searching for PR NF A88-110-1
searching for NF L54-225-002
searching for PR NF E05-031-607
logging in
logging in
logging in
logging in
searching for NF M08-011
searching for NF C28-913
searching for NF C54-145
searching for NF L43-615
logging in
logging in
logging in
logging in
searching for PR NF E29-695/A1
searching for NF F78-001
search

searching for NF C75-745-2-12
logging in
searching for NF A49-313-2
logging in
logging in
searching for NF C90-302-8/A1
logging in
searching for NF C60-436/A2
searching for NF G07-019-9
logging in
searching for NF S76-011-10
logging in
logging in
searching for NF E90-373-2
logging in
searching for NF C61-202/A1
searching for NF L14-306
logging in
searching for NF E31-404-1
logging in
logging in
logging in
searching for PR NF U34-270
searching for NF P10-202-4
searching for NF B54-022
logging in
searching for NF C93-024-2
logging in
logging in
searching for PR NF P98-880-6
logging in
searching for NF C48-2XXF2
searching for NF A95-771-2
logging in
searching for NF C53-040-1
searching for NF C43-814/A1
logging in
logging in
searching for NF P98-481-2
logging in
logging in
searching for NF G07-220-3
searching for NF E22-341
searching for NF C93-547-2-2
logging in
logging in
logging in
searching for PR NF C74-365
logging in
searching for PR NF P22-101-4
searching for NF X20-510-4
logging i

logging in
logging in
searching for NF EN 13565-1/IN1
searching for NF EN 1710/IN1
logging in
logging in
searching for NF T35-205-1
searching for NF EN 13546/IN1
logging in
logging in
searching for NF A89-234
logging in
searching for NF E35-413
logging in
searching for PR NF EN 62830-3
searching for NF C71-000/A1
logging in
logging in
searching for NF C47-732/A1
searching for NF S91-277
logging in
logging in
searching for NF L24-960
searching for NF P98-662
logging in
logging in
searching for NF Z84-489-26
logging in
searching for PR NF E58-170-3-1
searching for NF X06-500
logging in
logging in
searching for PR NF D90-503
searching for NF T37-001-12
logging in
logging in
searching for NF G08-019-2
searching for NF C96-013-6-17
logging in
logging in
searching for NF C32-027/A2
searching for NF G52-231-1
logging in
logging in
searching for NF Z84-500-1
searching for NF S95-121
logging in
logging in
searching for NF V03-143
searching for NF L00-097
logging in
logging in
searching for NF D

>> refreshing...
>> target: 89226, found: 60310, missing: 0
